In [1]:
import configparser
import errno
import json
import os
import pprint
import time
import typing
from datetime import datetime, timedelta, timezone

import requests

In [2]:
def get_conversations_history(
        token: str,
        channel: str,
        latest: float = 0,
        limit: int = 100):

    Conversations_History_API_URL = "https://slack.com/api/conversations.history"

    params = {
        'token': token,
        'channel': channel,
        "limit": limit,
        "latest": latest}
    r = requests.get(Conversations_History_API_URL, params=params)
    json_data = r.json()
    if json_data.get('ok', False) is False:
        print(json_data)
        raise Exception('FAILD TO GET CONVERSATIONS HISTORY!')
    else:
        return json_data


In [22]:
def retuen_timestanp_movedbyday(diff: int = -2):
    JST = timezone(timedelta(hours=+9), 'JST')
    td_two_days_before = timedelta(days=diff)
    today_at_midnight = datetime.now(JST).replace(
        hour=0, minute=0, second=0, microsecond=0)
    two_days_before = today_at_midnight + td_two_days_before
    return two_days_before.timestamp()

In [16]:
def delete_message(token: str, channel: str, ts: float):
    Chat_Delete_API_URL = "https://slack.com/api/chat.delete"

    params = {'token': token, 'channel': channel, "ts": ts}
    r = requests.get(Chat_Delete_API_URL, params=params)
    json_data = r.json()

    if json_data.get('ok', False) is False:
        print('message の削除に失敗')
        print(json_data)

In [17]:
config_ini = configparser.ConfigParser()
config_ini_path = 'config.ini'

if not os.path.exists(config_ini_path):
    raise FileNotFoundError(
        errno.ENOENT, os.strerror(
            errno.ENOENT), config_ini_path)

config_ini.read(config_ini_path, encoding='utf-8')

read_default = config_ini['DEFAULT']
Token = read_default.get('OAuth_Token')
CHANNEL_ID = read_default.get('CHANNEL_ID')
GOOGLE_CALENDER_ID = read_default.get('GOOGLE_CALENDER_ID')


In [39]:
json_data = get_conversations_history(
    Token, CHANNEL_ID, retuen_timestanp_movedbyday())

In [40]:
cnt = 0

for i in json_data["messages"]:
    if 'bot_id' in i:
        if i["bot_id"] == GOOGLE_CALENDER_ID:
            # delete_message(Token, CHANNEL_ID, i["ts"])
            cnt += 1
            pprint.pprint(i)
print(f"{cnt} msg del done")

{'attachments': [{'color': '92e1c0',
                  'fallback': '河村研ミーティングK: '
                              '<!date^1586739600^{date_short_pretty} from '
                              '{time}|April 12th, 2020 from 10:00 AM> to '
                              '<!date^1586746800^{time}|12:00 PM GMT+0900>',
                  'id': 1,
                  'mrkdwn_in': ['text', 'pretext'],
                  'text': '<!date^1586739600^{date_short_pretty} from '
                          '{time}|April 12th, 2020 from 10:00 AM> to '
                          '<!date^1586746800^{time}|12:00 PM GMT+0900> at '
                          '<https://zoom.us/j/703135382>',
                  'title': '河村研ミーティングK',
                  'title_link': 'https://www.google.com/calendar/event?eid=YTdia3VrbWNhNTgyZW8wajk3cTd1bmNmazhfMjAyMDA0MTNUMDAwMDAwWiBzaGluc2h1LXUuYWMuanBfMGIwMzU3b2RtYjBoYXNlazhoMWd0Y2RwMTBAZw'},
                 {'color': '92e1c0',
                  'fallback': 'KISS-MT(M2研究紹介): '
        

In [35]:
json_data

{'ok': True,
 'latest': '1586012400.000000',
 'messages': [{'client_msg_id': 'ed4ba7c9-6e40-4a66-8a71-bda963d19418',
   'type': 'message',
   'text': '月曜日6日10時から研究室全体のミーティングをネットで行います．\n各自ネットが使える環境で待機してください．',
   'user': 'UDGAH9AGP',
   'ts': '1585883699.001100',
   'team': 'TD9V1JA9Z',
   'blocks': [{'type': 'rich_text',
     'block_id': 'aE4+',
     'elements': [{'type': 'rich_text_section',
       'elements': [{'type': 'text',
         'text': '月曜日6日10時から研究室全体のミーティングをネットで行います．\n各自ネットが使える環境で待機してください．'}]}]}]},
  {'type': 'message',
   'subtype': 'channel_join',
   'ts': '1585879994.000200',
   'user': 'U011ALLTSEB',
   'text': '<@U011ALLTSEB>さんがチャンネルに参加しました'},
  {'type': 'message',
   'subtype': 'bot_add',
   'text': 'このチャンネルに次のインテグレーションを追加しました : <https://kawamlab.slack.com/services/BLK4D0PK6|河村研予定表>',
   'user': 'UDGAH9AGP',
   'bot_id': 'BLK4D0PK6',
   'bot_link': '<https://kawamlab.slack.com/services/BLK4D0PK6|河村研予定表>',
   'ts': '1585878904.000100'},
  {'type': 'message',
   'subtyp